<a href="https://colab.research.google.com/github/bingsen-wang/PowerElectronics/blob/main/ThermalImpedance_SinglePulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1><b>Thermal Impedance
</h1></b>

Dr. Bingsen Wang

12/29/2023
</center>

#Transient Thermal Impedance

In most datasheet of power switching devices, there is transient thermal impedance that depends on the pulse duration of the power losses. The concept of transient thermal impedance may be confusing since it seems that the thermal impedance varies as the power loss input changes. This document explains how the time-independent impance corresonds to the so called transient impedance.

The thermal impedance can be characterized with the Foster RC network.

$$Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{R_iC_is + 1}\tag{1}$$

Let the time constant $\tau_i$ for the $i^{th}$ RC network be
$$\tau_i = R_iC_i\tag{2}$$

Then the thermal impedance can be rewritten as
$$Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{\tau_is + 1}\tag{3}$$

The unit step response of the junction temperature (rise) is
$$T_j(s) = \frac{1}{s}Z_{th}(s) = \sum_{i=1}^{n}\frac{R_i}{\tau_is^2 + s}=\sum_{i=1}^{n}R_i\left(\frac{1}{s}-\frac{1}{s+{1\over\tau_i}}\right)\tag{4}$$

The transient junction temperature rise in time domain is
$$T_j(t) = \sum_{i=1}^{n}R_i\left(1- e^{-{t\over\tau_i}}\right)\tag{5}$$

The so-called transient thermal impedance essentially takes the junction temperature in response to unit step input as the impedance!

#Python Code
The Python code illustrates the real and reactive powers.

In [ ]:
import matplotlib
from os import linesep
from numpy import logspace,exp,log
import matplotlib.pyplot as plt
from scipy.integrate import quad
from matplotlib import animation,rc
rc('animation', html='jshtml')
plt.rcParams.update({"font.family" : "sans serif","mathtext.fontset" : "cm"})

def Tj(t,Rs,Cs):
  T=0
  for i,R in enumerate(Rs):
    tau = R*Cs[i]
    T += R*(1-exp(-t/tau))
  return T

#parameters
clst=['lightskyblue','violet','aqua','cornflowerblue','yellow','w',(.1,.1,.1)] #colors dark bg
# clst=['mediumblue','m','brown','c','r',(.1,.1,.1),'w'] #colors light bg
Nf = 280
t=logspace(-6,0,Nf)

#RC network
R_lst = [2.748817E-03, 5.715661E-03, 4.153561E-02, 5.616478E-02,
         0.3286516, 1.016057, 0.130071]
C_lst = [2.916343E-05, 1.725521E-04, 2.092143E-04, 1.786133E-03,
         2.129755E-03, 8.451135E-03, 0.0863404]
Zth = Tj(t,R_lst,C_lst)

fig = plt.figure(figsize=(16,9),facecolor=clst[-1])
ax_frame = [[0.08, .08, .9, .9], [1e-6,1], [1e-2,10],
            ]# [pos-boundary, xlim, ylim] for subplots 16:9

ax=fig.add_axes(ax_frame[0],xlim=ax_frame[1], ylim=ax_frame[2],fc='none') #no face color
ax1=fig.add_axes([.6,.1,.3,.5],xlim=[0,1], ylim=[0,1],fc=(.3,.3,.3)) #no face color
ax1.axis('off')
ax1.patch.set_visible(True)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'$t\ (s)$',size=20,color=clst[-2])
ax.set_ylabel(r'$Z_{th}\ (^\circ C/W)$',size=20,color=clst[-2])
ax.tick_params(labelcolor=clst[-2],labelsize=16)
ax.grid(True,which='major',color=clst[2], alpha=0.5)
ax.grid(True,which='minor',color=clst[-2], alpha=0.3)
ax.minorticks_on()
for spine in ax.spines.values():
  spine.set_edgecolor(clst[0])
line_Zth = ax.plot([],[],color=clst[1],lw=4)[0]
txt_Rs, txt_Cs = ([],[])
for i in range(len(R_lst)):
  txt_Rs.append(ax1.text(0,.9-.12*i,'$R_'+str(i+1)+'='+str('{:e}'.format(R_lst[i]))+'$',
                         color=clst[-2],size=16,alpha=0))
for i in range(len(C_lst)):
  txt_Cs.append(ax1.text(0.5,.9-.12*i,'$C_'+str(i+1)+'='+str('{:e}'.format(C_lst[i]))+'$',
                         color=clst[-2],size=16,alpha=0))
kk=int(Nf/(len(txt_Cs)+len(txt_Rs)))
# animation function. This is called sequentially
def animate(i):
  line_Zth.set_data(t[:i+1],Zth[:i+1])
  if i%kk==0:
    if int(i/kk)<len(txt_Rs):
      txt_Rs[int(i/kk)].set_backgroundcolor((.2,.2,.2))
      txt_Rs[int(i/kk)].set_alpha(1)
    else:
      txt_Cs[int(i/kk)-len(txt_Rs)].set_backgroundcolor((.2,.2,.2))
      txt_Cs[int(i/kk)-len(txt_Rs)].set_alpha(1)
  return
anim = animation.FuncAnimation(fig, animate, frames=Nf, interval=20)
# anim #uncomment to generate animation in the output area
# to save the animation, uncomment the following line
anim.save("ThermalImpedance_singleShot.mp4", fps=25, dpi = 120)